# Bind Tools

- Author: [Jaemin Hong](https://github.com/geminii01)
- Peer Review: 
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb)

## Overview

This tutorial introduces `bind_tools` , a powerful function in LangChain for integrating custom tools with LLMs.

It aims to demonstrate how to create, bind, and execute tools seamlessly, enabling enriched AI-driven workflows.

Through this guide, you'll learn to bind tools, parse and execute outputs, and integrate them into an `AgentExecutor` .

### Table of Contents

- [Overview](#overview)
- [Environement Setup](#environment-setup)
- [Tool Creation](#tool-creation)
- [Tool Binding](#tool-binding)
- [bind_tools + Parser + Execution](#bind_tools-+-parser-+-execution)
- [bind_tools to Agent & AgentExecutor](#bind_tools-to-agent-&-agentexecutor)

### References

- [Conceptual guide - Tool calling](https://python.langchain.com/docs/concepts/tool_calling/)
- [tool_calls](https://python.langchain.com/docs/concepts/tool_calling/#tool-calling-1)
- [AgentExecutor](https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain_community",
        "langchain_core",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "02-Bind-Tools",
    }
)

Environment variables have been set successfully.


You can alternatively set API keys such as `OPENAI_API_KEY` in a `.env` file and load them.

[Note] This is not necessary if you've already set the required API keys in previous steps.

In [4]:
# Load API keys from .env file
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Tool Creation

Define tools for experimentation:

- `get_word_length` : Returns the length of a word
- `add_function` : Adds two numbers
- `bbc_news_crawl` : Crawls BBC news and extracts main content

[Note]

- Use the `@tool` decorator for defining tools, and provide clear English docstrings.

In [5]:
import requests
from bs4 import BeautifulSoup
from langchain_core.tools import tool


# Define the tools
@tool
def get_word_length(word: str) -> int:
    """Return the length of the given text"""
    return len(word)


@tool
def add_function(a: float, b: float) -> float:
    """Add two numbers together"""
    return a + b


@tool
def bbc_news_crawl(news_url: str) -> str:
    """Crawl a news article from BBC"""
    response = requests.get(news_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract the desired information from the article
        article = soup.find("article")
        if article:
            title = article.find("h1").get_text()  # Extract the title
            content_list = [
                tag.get_text()
                for tag in article.find_all(["h2", "p"])
                if (tag.name == "h2" and "sc-518485e5-0" in tag.get("class", []))
                or (tag.name == "p" and "sc-eb7bd5f6-0" in tag.get("class", []))
            ]  # Extract the content
            content = "\n\n".join(content_list)
    else:
        print(f"HTTP request failed. Response code: {response.status_code}")
    return f"{title}\n\n----------\n\n{content}"


tools = [get_word_length, add_function, bbc_news_crawl]

## Tool Binding

Use the `bind_tools` function to bind the tools to an LLM model.

In [6]:
from langchain_openai import ChatOpenAI

# Create a model
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Tool binding
llm_with_tools = llm.bind_tools(tools)

Let's check the result!

The results are stored in `tool_calls` . Therefore, let's print `tool_calls` .

[Note]

- `name` is the name of the tool.
- `args` are the arguments passed to the tool.

In [7]:
# Execution result
llm_with_tools.invoke(
    "What is the length of the given text 'LangChain OpenTutorial'?"
).tool_calls

[{'name': 'get_word_length',
  'args': {'word': 'LangChain OpenTutorial'},
  'id': 'call_km7ieeNgjOvbPEfPt3bwO4cy',
  'type': 'tool_call'}]

Next, we connect `llm_with_tools` with `JsonOutputToolsParser` to parse `tool_calls` and review the results.

[Note]

- `type` is the name of the tool.
- `args` are the arguments passed to the tool.

In [8]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# Tool Binding + Tool Parser
chain = llm_with_tools | JsonOutputToolsParser(tools=tools)

# Execution Result
tool_call_results = chain.invoke(
    "What is the length of the given text 'LangChain OpenTutorial'?"
)
print(tool_call_results)

[{'args': {'word': 'LangChain OpenTutorial'}, 'type': 'get_word_length'}]


In [9]:
print(tool_call_results)
print("\n==========\n")

# First tool call result
single_result = tool_call_results[0]

print(single_result["type"])
print(single_result["args"])

[{'args': {'word': 'LangChain OpenTutorial'}, 'type': 'get_word_length'}]


get_word_length
{'word': 'LangChain OpenTutorial'}


Execute the tool matching the tool name.

In [10]:
tool_call_results[0]["type"], tools[0].name

('get_word_length', 'get_word_length')

The `execute_tool_calls` function identifies the appropriate tool, passes the corresponding `args` , and then executes the tool.

In [11]:
def execute_tool_calls(tool_call_results):
    """
    Function to execute the tool call results.

    :param tool_call_results: List of the tool call results
    :param tools: List of available tools
    """

    # Iterate over the list of the tool call results
    for tool_call_result in tool_call_results:
        # Tool name (function name)
        tool_name = tool_call_result["type"]
        # Tool arguments
        tool_args = tool_call_result["args"]

        # Find the tool that matches the name and execute it
        # Use the next() function to find the first matching tool
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)
        if matching_tool:
            # Execute the tool
            result = matching_tool.invoke(tool_args)
            print(
                f"[Executed Tool] {tool_name} [Args] {tool_args}\n[Execution Result] {result}"
            )
        else:
            print(f"Warning: Unable to find the tool corresponding to {tool_name}.")


# Execute the tool calls
execute_tool_calls(tool_call_results)

[Executed Tool] get_word_length [Args] {'word': 'LangChain OpenTutorial'}
[Execution Result] 22


## bind_tools + Parser + Execution

This time, the entire process will be executed in one step.

- `llm_with_tools` : The LLM model with bound tools
- `JsonOutputToolsParser` : The parser that processes the results of tool calls
- `execute_tool_calls` : The function that executes the results of tool calls

[Flow Summary]

1. Bind tools to the model
2. Parse the results of tool calls
3. Execute the results of tool calls

In [12]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# bind_tools + Parser + Execution
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls

In [13]:
# Execution Result 1
chain.invoke("What is the length of the given text 'LangChain OpenTutorial'?")

[Executed Tool] get_word_length [Args] {'word': 'LangChain OpenTutorial'}
[Execution Result] 22


In [14]:
# Execution Result 2
chain.invoke("114.5 + 121.2")

# Double check
print(114.5 + 121.2)

[Executed Tool] add_function [Args] {'a': 114.5, 'b': 121.2}
[Execution Result] 235.7
235.7


In [15]:
# Execution Result 3
chain.invoke("Crawl the news article: https://www.bbc.com/news/articles/cew52g8p2lko")

[Executed Tool] bbc_news_crawl [Args] {'news_url': 'https://www.bbc.com/news/articles/cew52g8p2lko'}
[Execution Result] New AI hub 'to create 1,000 jobs' on Merseyside

----------

A new Artificial Intelligence (AI) hub planned for Merseyside is set to create 1,000 jobs over the next three years, the government said.

Prime Minister Sir Keir Starmer said he wanted to make the UK one of the world's AI "super powers" as a way of boosting economic growth and improving public services.

Global IT company Kyndryl announced it was going to create the new tech hub in the Liverpool City Region.

Metro Mayor Steve Rotheram welcomed the investment, saying it would be "hugely beneficial" to the area.

'International investment'

In a speech setting out the government's AI ambitions, Starmer spoke of its "vast potential" for rejuvenating public services.

The government said its AI Opportunities Action Plan was backed by leading tech firms, some of which have committed £14bn towards various projec

## bind_tools to Agent & AgentExecutor

`bind_tools` provides schemas (tools) that can be used by the model.

`AgentExecutor` creates an execution loop for tasks such as invoking the LLM, routing to the appropriate tool, executing it, and re-invoking the model.

[Note]

- `Agent` and `AgentExecutor` will be covered in detail in the *next chapter* .

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# Create an Agent prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Create a model
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [17]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Use the tools defined previously
tools = [get_word_length, add_function, bbc_news_crawl]

# Create an Agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

Let's try calculating the length of the word.

In [18]:
# Execute the Agent
result = agent_executor.invoke(
    {"input": "What is the length of the given text 'LangChain OpenTutorial'?"}
)

# Execution Result
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'LangChain OpenTutorial'}`


22The length of the text "LangChain OpenTutorial" is 22 characters.

> Finished chain.
The length of the text "LangChain OpenTutorial" is 22 characters.


Let's try calculating the result of two numbers.

In [19]:
# Execute the Agent
result = agent_executor.invoke({"input": "Calculate the result of 114.5 + 121.2"})

# Execution Result
print(result["output"])
print("\n==========\n")
print(114.5 + 121.2)



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7The result of 114.5 + 121.2 is 235.7.

> Finished chain.
The result of 114.5 + 121.2 is 235.7.


235.7


Let's try adding more than two numbers. 

In this process, you can observe that the agent verifies its own results and repeats the process if necessary.

In [20]:
# Execute the Agent
result = agent_executor.invoke(
    {"input": "Calculate the result of 114.5 + 121.2 + 34.2 + 110.1"}
)

# Execution Result
print(result["output"])
print("\n==========\n")
print(114.5 + 121.2 + 34.2 + 110.1)



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7
Invoking: `add_function` with `{'a': 235.7, 'b': 34.2}`


269.9
Invoking: `add_function` with `{'a': 34.2, 'b': 110.1}`


144.3
Invoking: `add_function` with `{'a': 269.9, 'b': 110.1}`


380.0The result of adding 114.5, 121.2, 34.2, and 110.1 is 380.0.

> Finished chain.
The result of adding 114.5, 121.2, 34.2, and 110.1 is 380.0.


380.0


Let's try summarizing the news article.

In [21]:
# Execute the Agent
result = agent_executor.invoke(
    {
        "input": "Summarize the news article: https://www.bbc.com/news/articles/cew52g8p2lko"
    }
)

# Execution Result
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `bbc_news_crawl` with `{'news_url': 'https://www.bbc.com/news/articles/cew52g8p2lko'}`


New AI hub 'to create 1,000 jobs' on Merseyside

----------

A new Artificial Intelligence (AI) hub planned for Merseyside is set to create 1,000 jobs over the next three years, the government said.

Prime Minister Sir Keir Starmer said he wanted to make the UK one of the world's AI "super powers" as a way of boosting economic growth and improving public services.

Global IT company Kyndryl announced it was going to create the new tech hub in the Liverpool City Region.

Metro Mayor Steve Rotheram welcomed the investment, saying it would be "hugely beneficial" to the area.

'International investment'

In a speech setting out the government's AI ambitions, Starmer spoke of its "vast potential" for rejuvenating public services.

The government said its AI Opportunities Action Plan was backed by leading tech firms, some of which have committed £14bn to